## 🔧 Setup

The script begins by setting paths to input CSV files (countries, data, and indicator metadata) and defining constants like the minimum required year and number of top countries to consider.

---

## 🧠 Indicator Stacks

Indicators are grouped into four thematic categories (called "stacks"):

* **Tertiary Demand**: Measures related to higher education participation (e.g. enrolment, graduation).
* **Digital Access**: Metrics around internet and computer access.
* **Demographics**: Focuses on the youth population, a key target for online education.
* **Purchasing Power**: Economic indicators such as GDP and GNI per capita.

Each stack is defined by a set of keywords used to identify relevant indicators.

---

## 🔍 Indicator Extraction

The script loads the indicator metadata and searches for matches based on the defined keywords. It optionally includes topic-level filters for stacks like "tertiary demand."

The result is a curated list of indicator codes and names for each stack, to be used in later analysis.

In [ ]:
# ------------------------------------------------------------------
# 0. Paths & parameters
# ------------------------------------------------------------------
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import geopandas as gpd

DATA_DIR   = Path("./Projet+Python_Dataset_Edstats_csv")
COUNTRIES  = DATA_DIR / "EdStatsCountry.csv"
DATA       = DATA_DIR / "EdStatsData.csv"
INDICATORS = DATA_DIR / "EdStatsSeries.csv"

MIN_REQUIRED = 2000
TOP_N        = 50

# 1. The four keyword stacks
STACKS = {
    "tertiary_demand": [                         # (a) demand / participation
        r"\bTertiary\b", r"Post[- ]Secondary", r"\bISCED 4\b",
        r"Gross enrolment", r"Graduat", r"Completion",
    ],
    "digital_access": [                          # (b) digital access & ICT
        r"Internet users", r"Personal computers",
    ],
    "demographics": [                            # (c) demographics of learners
        r"Population, ages 15-24",
    ],
    "purchasing_power": [                        # (d) purchasing power, macro
        r"GDP per capita", r"GNI per capita",
    ],
}

TOPIC_EXTRA = {                                  # topic strings that matter
    "tertiary_demand": r"Tertiary|Post-Secondary/Non-Tertiary",
}

# 2. Build the hit-lists for each stack
series_df = pd.read_csv(INDICATORS, low_memory=False)

stack_hits = {}

for stack, kw_list in STACKS.items():
    pat_name   = "|".join(kw_list)
    mask_name  = series_df["Indicator Name"].str.contains(
        pat_name, case=False, regex=True, na=False
    )
    if stack in TOPIC_EXTRA:
        mask_topic = series_df["Topic"].str.contains(
            TOPIC_EXTRA[stack], case=False, regex=True, na=False
        )
        mask = mask_name | mask_topic
    else:
        mask = mask_name

    hits = (
        series_df.loc[mask, ["Series Code", "Indicator Name"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    stack_hits[stack] = hits

## 📊 Indicator Coverage Analysis & Filtering

This section of the script ensures that only well-populated indicators are kept for further analysis by counting and filtering based on data availability.

---

### 🔢 Step 3: Count Available Data Points per Indicator

The script loads the full dataset and counts how many actual (non-missing) yearly values exist for each row. It then aggregates these counts by indicator to compute the **total number of valid data points per indicator**.

---

### ✅ Step 4: Select High-Coverage Indicators

For each indicator stack (e.g., demand, access, demographics), the script retains only the indicators that meet a minimum threshold of data availability (`MIN_REQUIRED`). This helps ensure reliability and comparability across countries.

A summary is printed showing:

* How many indicators were evaluated.
* How many met the threshold.
* The retention percentage.

---

### 🧹 Step 5: Filter the Full Dataset

Finally, the script filters the full data table to keep **only the high-coverage indicators** selected in the previous step. This results in a cleaned dataset (`data_strong`) focused on meaningful and complete indicators, ready for scoring or visualization.

In [ ]:
# 3. Count real data cells for every indicator
data_df = pd.read_csv(DATA, low_memory=False)
YEAR_COLS = data_df.columns[4:]

data_df["n_values_row"] = data_df[YEAR_COLS].notna().sum(axis=1)

# Function to roll counts up per indicator
def get_indicator_counts(codes: pd.Series) -> pd.DataFrame:
    tmp = data_df[data_df["Indicator Code"].isin(codes)]
    counts = (
        tmp.groupby(["Indicator Code", "Indicator Name"], as_index=False)
        ["n_values_row"].sum()
        .rename(columns={"n_values_row": "total_values"})
        .sort_values("total_values", ascending=False)
    )
    return counts

stack_counts = {
    stack: get_indicator_counts(hits["Series Code"])
    for stack, hits in stack_hits.items()
}

# 4. Decide which indicators to keep per stack
stack_best = {}

for stack, counts in stack_counts.items():
    if MIN_REQUIRED:
        keep = counts[counts["total_values"] >= MIN_REQUIRED]
    else:
        keep = counts.head(TOP_N)
    stack_best[stack] = keep

    print(f"\n=== {stack.replace('_', ' ').title()} ===")
    print(keep.to_string(index=False))
    print(
        f"Kept {len(keep):,} of {len(counts):,} indicators "
        f"({len(keep)/max(len(counts),1):.1%}) with "
        f"{'≥'+str(MIN_REQUIRED) if MIN_REQUIRED else 'Top-'+str(TOP_N)} values."
    )

# 5. Filter the big data table to the kept indicators only
kept_codes = pd.concat([df["Indicator Code"] for df in stack_best.values()]).unique()
data_strong = data_df[data_df["Indicator Code"].isin(kept_codes)].copy()


## ✅ Step 6: Final Indicator Shortlist

This step manually defines a curated list of key indicators to be used in the final analysis.

---

### 🎯 Purpose

From the previously filtered high-coverage indicators, a small set of the **most relevant and interpretable indicators** is selected for each thematic category. These were chosen for their clarity, availability, and direct relevance to assessing the potential for online post-secondary education expansion.

---

### 📚 Final Indicator Groups

* **Tertiary Demand**
  Indicators measuring enrolment levels and the size of the post-secondary age group.

* **Digital Access**
  A core metric reflecting internet penetration among the population.

* **Demographics**
  Focused on the size of the youth population (ages 15–24), the primary target audience.

* **Purchasing Power**
  Includes GDP and GNI per capita, representing economic capability at the individual level.

In [ ]:
# 6. Define the final indicator short-list
INDICATOR_CANDIDATES = {
    "tertiary_demand": [
        "SE.TER.ENRR",       # Gross enrolment ratio, tertiary (%)
        "SP.TER.TOTL.IN",    # Population of tertiary-age cohort (number)
        "UIS.TE_100000.56",  # Tertiary enrolment per 100 000 population
    ],
    "digital_access": [
        "IT.NET.USER.P2",    # Internet users per 100 people
    ],
    "demographics": [
        "SP.POP.1524.TO.UN", # Population aged 15-24 (total)
    ],
    "purchasing_power": [
        "NY.GDP.PCAP.PP.KD", # GDP pc, PPP, constant 2011 $
        "NY.GNP.PCAP.CD",    # GNI pc, Atlas method, current $
    ],
}

## 🧭 Step 7: Visualize Missing Data

This step generates a heatmap to provide a **visual overview of missing values** in the dataset.

---

### 🔍 Purpose

By plotting a missing-value matrix for the entire dataset, this heatmap helps quickly identify:

* Patterns or clusters of missing data across years or indicators.
* Indicators or countries with sparse data.
* Potential structural gaps (e.g., certain years consistently missing).

---

### 📊 Output

The heatmap displays rows as data entries and columns as features. Missing values are highlighted, making data quality issues easy to spot before analysis continues.

In [ ]:
# 7. Display heatmap missing values
plt.figure(figsize=(20, 10))
sns.heatmap(data_df.isnull(), cbar=False,
            yticklabels=False)
plt.title("Missing-value map — EdStatsData")
plt.tight_layout()
plt.show()

## 📥 Step 8: Load and Filter Final Data

The script reads the main education dataset in chunks, keeping only:

* The selected shortlist of high-quality indicators.
* Years of interest: historical (2000–2015) and forecast (2020–2040).
* Countries listed in the metadata.

The result is a cleaned dataset (`raw`) containing only the relevant rows and columns for analysis.

---

## 🧮 Step 9: Extract Most Recent Values

For each indicator concept (e.g. digital access, demographics), the script:

* Selects the most complete indicator (best coverage).
* Uses the latest available **historical value**, or falls back to the **earliest forecast** if needed.
* Flags whether the value came from a forecast.

The result is a tidy table (`tidy`) with one row per country per concept.

---

## 🔁 Step 10: Collapse to One Value per Concept

The script reshapes the tidy table into a wide format with one row per country and one column per concept. It also merges country metadata like region and income group.

---

## 📊 Step 11: Score Normalization & Composite Index

Each concept is normalized using min-max scaling:

* Higher scores = better conditions (e.g. large youth population, strong digital access).
* For **enrolment**, lower values imply higher market gaps (and opportunity), so the score is reversed.

A weighted formula then combines these into a single **potential score**, indicating each country’s suitability for online post-secondary education expansion.

---

## 🏁 Step 12: Ranking & Output

Countries are ranked by their potential score and saved to a CSV file for further use. This final ranked list identifies the most promising markets based on data coverage, need, and infrastructure readiness.

In [ ]:
# 8. Read & filter data
YEAR_MIN, YEAR_MAX  = 2000, 2015
FCAST_MIN, FCAST_MAX = 2020, 2040

codes_flat = {code for lst in INDICATOR_CANDIDATES.values() for code in lst}
years_keep = {str(y) for y in range(YEAR_MIN,  YEAR_MAX+1)} | \
             {str(y) for y in range(FCAST_MIN, FCAST_MAX+1)}

keep_cols = lambda c: (
        c in ["Country Code", "Indicator Code"]
        or (c.isdigit() and c in years_keep)
)

countries   = pd.read_csv(COUNTRIES, low_memory=False)
country_set = set(countries["Country Code"])

parts = []
for chunk in pd.read_csv(DATA, chunksize=30_000, usecols=keep_cols, low_memory=False):
    parts.append(
        chunk[
            chunk["Indicator Code"].isin(codes_flat)
            & chunk["Country Code"].isin(country_set)
            ]
    )
raw = pd.concat(parts, ignore_index=True)

year_cols  = raw.columns[raw.columns.str.isdigit()]

hist_cols  = [c for c in year_cols if YEAR_MIN  <= int(c) <= YEAR_MAX]
fcast_cols = [c for c in year_cols if FCAST_MIN <= int(c) <= FCAST_MAX]

# 9. Helperlatest observed or earliest forecast
def latest_or_forecast(df_hist, df_fcast):
    hist = (df_hist.reindex(sorted(df_hist.columns, key=int, reverse=True), axis=1)
            .apply(pd.to_numeric, errors="coerce")
            .bfill(axis=1).iloc[:, 0])

    if df_fcast.shape[1]:
        fcast = (df_fcast.reindex(sorted(df_fcast.columns, key=int), axis=1)
                 .apply(pd.to_numeric, errors="coerce")
                 .ffill(axis=1).iloc[:, -1])
    else:
        fcast = pd.Series(np.nan, index=df_hist.index)

    value = hist.fillna(fcast)
    flag  = (hist.isna() & value.notna()).astype(int)
    return value, flag

# 10. Collapse to one best indicator per concept
tidy_blocks = []

for concept, codes in INDICATOR_CANDIDATES.items():
    coverage = {
        code: raw.loc[raw["Indicator Code"] == code, hist_cols].notna().any(axis=1).sum()
        for code in codes
    }
    best_code = max(coverage, key=coverage.get)

    block = raw[raw["Indicator Code"] == best_code].set_index("Country Code")
    val, flag = latest_or_forecast(block[hist_cols], block[fcast_cols])

    tidy_blocks.append(
        pd.DataFrame({
            "Country Code": block.index,
            "concept"     : concept,
            "value"       : val.values,
            "forecast"    : flag.values,
        })
    )

tidy = pd.concat(tidy_blocks, ignore_index=True)

# 11. Wide table  +  composite score
wide = (tidy.pivot(index="Country Code", columns="concept", values="value")
        .reset_index()
        .merge(countries, on="Country Code", how="left"))

# ─── rescale helpers ──────────────────────────────────────────────
def minmax(s):
    v = s.dropna()
    if v.empty: return pd.Series(np.nan, index=s.index)
    lo, hi = v.min(), v.max()
    if lo == hi: return pd.Series(0.5, index=s.index)
    return (s - lo) / (hi - lo)

# individual scores
wide["size_score"]     = minmax(wide["demographics"])
wide["access_score"]   = minmax(wide["digital_access"])
tertiary_scaled        = minmax(wide["tertiary_demand"])
wide["gap_score"]      = 1 - tertiary_scaled          # low enrolment ⇒ big gap
wide["wealth_score"]   = minmax(wide["purchasing_power"])

weights = dict(size=0.30, access=0.25, gap=0.25, wealth=0.20)

wide["potential_score"] = (
        weights["size"]   * wide["size_score"]
        + weights["access"] * wide["access_score"]
        + weights["gap"]    * wide["gap_score"]
        + weights["wealth"] * wide["wealth_score"]
)

# 12. Rank & plot
ranking = (
    wide[["Country Code", "Short Name", "Region", "Income Group",
          "potential_score"]]
    .dropna(subset=["potential_score"])
    .sort_values("potential_score", ascending=False)
    .reset_index(drop=True)
)

ranking.to_csv("country_potential_scores_final.csv", index=False)
print(f"✅ Saved {ranking.shape[0]} ranked countries to CSV")


## 📊 Visualization & Insights

This section visualizes the final results of the scoring model, helping interpret the rankings and understand the underlying drivers of each country’s potential for online school expansion.

---

### 🏆 1. Top Countries Bar Chart

Displays the top 20 countries ranked by **potential score** in a horizontal bar chart.
This provides a quick visual of the highest-scoring markets for online education expansion.

---

### 🔄 2. Score Component Correlation Heatmap

A correlation matrix shows how the different **score components** relate to one another (e.g., whether internet access is correlated with purchasing power).
This helps identify dependencies or redundancies in the scoring dimensions.

---

### 🗺️ 3. World Map: Geospatial Distribution

A choropleth map visualizes the **geographic distribution** of potential scores.
Top-ranking countries are annotated directly on the map, highlighting regions with the strongest opportunity signals.

---

### 🧩 4. Score Breakdown: Normalized vs Raw Values

This dual-panel visualization shows:

* **Left Panel:** Normalized component scores (0–1 scale) for each top country, illustrating how each dimension contributes to the final potential score.
* **Right Panel:** Corresponding **raw indicator values** (on a log scale) to understand the actual values behind the scores, correcting for skewed distributions.

Together, these plots offer a comprehensive view of both **who ranks highest** and **why**.

In [ ]:
TOP_N = 20
ranking = wide[["Country Code", "Short Name", "Region", "Income Group", "potential_score"]].copy()
ranking = ranking.dropna(subset=["potential_score"]).sort_values("potential_score", ascending=False)

# 1. Top 20 Countries Plot
plt.figure(figsize=(10, 8))
topN = ranking.head(TOP_N)
sns.barplot(data=topN, x="potential_score", y="Short Name", hue="Short Name", dodge=False, legend=False)
plt.title("Top 20 Countries for Online School Expansion")
plt.xlabel("Potential Score")
plt.ylabel("Country")
plt.tight_layout()
plt.show()

# 2. Score Correlation Heatmap
score_cols = ["size_score", "access_score", "gap_score", "wealth_score", "potential_score"]
plt.figure(figsize=(8, 6))
sns.heatmap(wide[score_cols].corr(), annot=True, cmap="coolwarm", center=0)
plt.title("Correlation Between Score Components")
plt.tight_layout()
plt.show()

# 3. World Geo Map of Potential Score
world = gpd.read_file("./ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
world = world.rename(columns={"NAME": "Short Name"})
world_data = world.merge(ranking, on="Short Name", how="left")

top_countries = ranking.sort_values("potential_score", ascending=False).head(TOP_N)["Short Name"]

fig, ax = plt.subplots(figsize=(15, 10))
world_data.plot(
    column="potential_score",
    ax=ax,
    cmap="YlGnBu",
    legend=True,
    legend_kwds={"label": "Potential Score", "shrink": 0.5},
    edgecolor="gray",
    missing_kwds={"color": "lightgray", "label": "No data"}
)
ax.set_title(f"Top {TOP_N} Countries for Online School Expansion (Age 17+)")
ax.axis("off")

# Annotate only the top countries
for _, row in world_data[world_data["Short Name"].isin(top_countries)].iterrows():
    try:
        centroid = row["geometry"].centroid
        ax.text(centroid.x, centroid.y, row["Short Name"], fontsize=6,
                ha='center', va='center', color='black')
    except:
        continue

plt.show()

# 4. Score Breakdown – Top N Countries
components = ["size_score", "access_score", "gap_score", "wealth_score"]
raws = ["demographics", "digital_access", "tertiary_demand", "purchasing_power"]
component_labels = ["Youth Pop (score)", "Internet (score)", "Enrolment Gap (score)", "Wealth (score)"]
raw_labels = ["Youth Pop (raw)", "Internet (raw)", "Enrolment (%)", "GDP/GNI"]

top10 = wide[wide["Country Code"].isin(ranking.head(TOP_N)["Country Code"])].copy()
melted_scores = top10.melt(id_vars=["Short Name"], value_vars=components, var_name="Component", value_name="Score")
melted_scores["Component"] = melted_scores["Component"].map(dict(zip(components, component_labels)))

melted_raws = top10.melt(id_vars=["Short Name"], value_vars=raws, var_name="Component", value_name="Raw Value")
melted_raws["Component"] = melted_raws["Component"].map(dict(zip(raws, raw_labels)))

fig, axes = plt.subplots(1, 2, figsize=(18, 8), gridspec_kw={"width_ratios": [1.2, 1]})

# Normalized score barplot
sns.barplot(data=melted_scores, x="Score", y="Short Name", hue="Component", ax=axes[0])
axes[0].set_title("Score Component Breakdown (Normalized)")
axes[0].set_xlabel("Score (0–1)")
axes[0].legend(loc="lower right")
axes[0].xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Raw values barplot (use log scale)
sns.barplot(data=melted_raws, x="Raw Value", y="Short Name", hue="Component", ax=axes[1])
axes[1].set_xscale("log")  # Apply log scale to fix range imbalance
axes[1].set_title("Raw Indicator Values (Log Scale)")
axes[1].set_xlabel("Raw Value (log scale)")
axes[1].legend(loc="lower right")

axes[1].xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f"{int(x):,}"))

plt.tight_layout()
plt.show()